In [1]:
import os, sys

project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
os.environ['PYTHONPATH'] = project_root
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("PYTHONPATH manually set to:", os.environ['PYTHONPATH'])

PYTHONPATH manually set to: c:\Users\ndhaj\Desktop


## Holography

In [5]:
import numpy as np
from gpie import Graph, SupportPrior, ConstWave, fft2, AmplitudeMeasurement, pmse
from gpie.core.linalg_utils import circular_aperture, masked_random_array

# ==== Setting Parameter ====
H, W = 256, 256
shape = (H, W)
var = 1e-4
rng = np.random.default_rng(seed=42)
support_x = circular_aperture(shape=shape, radius=0.2, center=(-0.2, -0.2))
data_x = masked_random_array(support_x, dtype=np.complex128, rng=rng)
support_y =  circular_aperture(shape=shape, radius=0.2, center=(0.2, 0.2))

# ==== Graph ====
class HolographicCDI(Graph):
    def __init__(self):
        super().__init__()
        # known and unknown wave
        X = ~ConstWave(data_x, label="X")
        Y = ~SupportPrior(support = support_y, label="Y")
        # diffraction measurement
        with self.observe():
            meas = AmplitudeMeasurement(var = var, damping = 0.1) @ (fft2(X + Y))
        self.compile()

# ==== グラフ構築・サンプル生成 ====
g = HolographicCDI()
g.set_init_rng(np.random.default_rng(seed=12))
g.generate_sample(rng=np.random.default_rng(seed=99), update_observed=True)

true_x = g.get_wave("X").get_sample()
true_y = g.get_wave("Y").get_sample()

# ==== 推論ループ ====
def monitor(graph, t):
    y = graph.get_wave("Y").compute_belief().data
    err = pmse(y, true_y)
    if t % 100 == 0:
        print(f"[t={t}] PMSE (sum) = {err:.5e}")

g.run(n_iter=500, callback=monitor, verbose = False)


[t=0] PMSE (sum) = 6.93560e-01
[t=100] PMSE (sum) = 5.44096e-04
[t=200] PMSE (sum) = 5.44095e-04
[t=300] PMSE (sum) = 5.44095e-04
[t=400] PMSE (sum) = 5.44095e-04


In [3]:
g.visualize()

Loading BokehJS ...

In [6]:
import numpy as np
from gpie import Graph, SupportPrior, fft2, AmplitudeMeasurement, pmse
from gpie.core.linalg_utils import circular_aperture, masked_random_array

# ==== Setting Parameters ====
H, W = 256, 256
shape = (H, W)
var = 1e-4
rng = np.random.default_rng(seed=42)

# known & unknown wave generation
support_x = circular_aperture(shape=shape, radius=0.2, center=(-0.2, -0.2))
data_x = masked_random_array(support_x, dtype=np.complex128, rng=rng)

support_y = circular_aperture(shape=shape, radius=0.2, center=(0.2, 0.2))


# ==== Graph ====
class HolographicCDI(Graph):
    def __init__(self):
        super().__init__()

        # Y is unknown; X is added as a constant array
        Y = ~SupportPrior(support=support_y, label="Y")
        with self.observe():
            meas = AmplitudeMeasurement(var=var, damping=0.1) @ (fft2(data_x + Y))  # ← ndarray + Wave triggers AddConstPropagator

        self.compile()


# ==== Run Inference ====
g = HolographicCDI()
g.set_init_rng(np.random.default_rng(seed=12))
g.generate_sample(rng=np.random.default_rng(seed=99), update_observed=True)

true_y = g.get_wave("Y").get_sample()


def monitor(graph, t):
    y = graph.get_wave("Y").compute_belief().data
    err = pmse(y, true_y)
    if t % 100 == 0:
        print(f"[t={t}] PMSE (sum) = {err:.5e}")


g.run(n_iter=500, callback=monitor, verbose=False)


[t=0] PMSE (sum) = 4.21300e-01
[t=100] PMSE (sum) = 2.78487e-04
[t=200] PMSE (sum) = 2.78494e-04
[t=300] PMSE (sum) = 2.78494e-04
[t=400] PMSE (sum) = 2.78494e-04


In [7]:
%prun g.run(n_iter=500, callback=monitor, verbose=False)

[t=0] PMSE (sum) = 2.78494e-04
[t=100] PMSE (sum) = 2.78494e-04
[t=200] PMSE (sum) = 2.78494e-04
[t=300] PMSE (sum) = 2.78494e-04
[t=400] PMSE (sum) = 2.78494e-04
 

         261660 function calls in 7.202 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1500    1.535    0.001    1.552    0.001 uncertain_array.py:333(__truediv__)
     2000    1.158    0.001    1.159    0.001 _pocketfft.py:51(_raw_fft)
      500    0.575    0.001    0.582    0.001 metrics.py:40(phase_align)
      500    0.529    0.001    0.535    0.001 uncertain_array.py:309(__mul__)
      500    0.501    0.001    0.551    0.001 amplitude_measurement.py:111(approximate_posterior)
     2000    0.481    0.000    0.488    0.000 numeric.py:1185(roll)
      500    0.476    0.001    2.334    0.005 fft_2d_propagator.py:112(compute_belief)
      500    0.339    0.001    0.342    0.001 uncertain_array.py:366(damp_with)
      500    0.269    0.001    0.423    0.001 uncertain_array_tensor.py:197(combine)
      500    0.188    0.000    0.212    0.000 metrics.py:3(mse)
     3500    0.179    0.000    0.179    0.000 {method 'reduce